# Regressão Linear

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

Os dados já estarão carregados e explorados. Será necessário apenas analisar os gráficos.

Quando o histograma for uma normal assimétrica para direita (tombada para esquerda), é bom trabalhar com o log dos valores.

## Treinamento e teste

In [ ]:
from sklearn.model_selection import train_test_sĺit
tran_set, test_set = train_test_split(dataset, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
print('{} train + {} test'.format(len(train_set), len(test_set)))

## Treinamento e teste estratificados

A separação pode levar em conta as colunas existentes ou pode fazer feature engineering para criar um novo critério. No caso do exemplo, é feito feature engineering.

In [ ]:
# Constroi uma coluna nova com categorias de renda fictícias.
dataset['income_cat'] = np.ceil(dataset['median_income'] / 1.5)
dataset['income_cat'].where(dataset['income_cat'] < 5, 5.0, inplace=True)


# Divide, de modo estratificado, o conjunto de dados.
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
for train_index, test_index in split.split(dataset, dataset["income_cat"]):
    strat_train_set = dataset.loc[train_index]
    strat_test_set = dataset.loc[test_index]

In [ ]:
# Remove a coluna nova, que foi adicionada apenas temporariamente.
strat_train_set.drop(['income_cat'], axis=1, inplace=True)
strat_test_set.drop(['income_cat'], axis=1, inplace=True)

In [ ]:
strat_train_set.info()

## {{EXPLORAÇÃO}}

In [ ]:
correlation_matrix = dataset.corr()

In [ ]:
correlation_matrix["median_house_value"].sort_values(ascending=False)

## Defina a métrica para calcular a qualidade do modelo

## Separando X e y

In [ ]:
# Variáveis independentes: dataset original menos a coluna de valores dependentes.
dataset_features = strat_train_set.drop("median_house_value", axis=1)

# Variável dependente, também chamada de label.
dataset_target = strat_train_set["median_house_value"].copy()

## Separando dados categoricos e numericos

In [ ]:
numeric_dataset = dataset_features.select_dtypes(include=['float64', 'int64'])
categ_dataset = dataset_features.select_dtypes(include=['object'])

## Criando transformadores

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    # Column index.
    rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6
    
    def __init__(self, add_bedrooms_per_room=True):  # No *args or **kwargs.
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self  # Nothing else to do.

    def transform(self, X, y=None):
        rooms_per_household = \
            X[:, CombinedAttributesAdder.rooms_ix] / X[:, CombinedAttributesAdder.household_ix]
        population_per_household = \
            X[:, CombinedAttributesAdder.population_ix] / X[:, CombinedAttributesAdder.household_ix]

        if self.add_bedrooms_per_room:
            bedrooms_per_room = \
                X[:, CombinedAttributesAdder.bedrooms_ix] / X[:, CombinedAttributesAdder.rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(dataset.values)

# Transformando em DataFrame, porque DataFrames são mais amigáveis.
columns_housing_extra_attribs = list(housing.columns) + ["rooms_per_household", "population_per_household"]
housing_extra_attribs = pd.DataFrame(housing_extra_attribs, columns=columns_housing_extra_attribs)
housing_extra_attribs.head()

## Aplicando transformadores isoladamente

### Caso dos polinomiais, logaritmos, etc

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import ElasticNet, Lasso, Ridge

# Nota: include_bias=False porque o bias (termo constante "1") já estará incluso no regressor linear.
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)
# Deve ser aplicado a cada feature que apresenta comportamento polinomial.
# O mesmo ocorre com outros tipos de funções

Compare os resultados obtidos aqui com aqueles da equação normal obtidos anteriormente.

Assim como o uso de *features* polinomiais leva a um modelo com melhor ajuste, podemos também usar outras funções como *features*, tais como $\text{log}()$, $\text{sin}()$, etc.

In [ ]:
# Experimento 2: grau adequdo.
poly_reg_2 = Pipeline([("poly_features",
                        PolynomialFeatures(degree=2, include_bias=False)),
                       ("std_scaler", StandardScaler()),
                       ("lin_reg", LinearRegression())]) ## Aqui poderia ser ElasticNet, Lasso, Ridge

"""
poly_reg_ridge = Pipeline([
    ("poly_features", PolynomialFeatures(degree=30, include_bias=False)),
    ("std_scaler", StandardScaler()),
    ("lin_reg", Ridge(alpha=alpha)),
])

poly_reg_lasso = Pipeline([
    ("poly_features", PolynomialFeatures(degree=30, include_bias=False)),
    ("std_scaler", StandardScaler()),
    ("lin_reg", Lasso(alpha=alpha)),
])

poly_reg_elasticnet = Pipeline([
    ("poly_features", PolynomialFeatures(degree=30, include_bias=False)),
    ("std_scaler", StandardScaler()),
    ("lin_reg", ElasticNet(alpha=alpha, l1_ratio=0.5, random_state=RAND_SEED)),
])
"""

poly_reg_2.fit(X, y)
y_test = poly_reg_2.predict(X_test.reshape(-1, 1))
plt.plot(X_test, y_test, 'b-', label='grau 2')

### Outros casos

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import OrdinalEncoder ## Se não for ordinal, não deve-se usar
from sklearn.preprocessing import OneHotEncoder

### Functional transformers

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import numpy as np

# Suponha que você tenha um DataFrame pandas chamado 'data' com a coluna 'feature' que deseja aplicar o log.
# Substitua 'data' e 'feature' pelo seu DataFrame e nome da coluna.

# Define a função que aplicará o log
def log_transform(X):
    return np.log(X)

# Crie um transformador de função
log_transformer = FunctionTransformer(func=log_transform, validate=False)

# Crie sua pipeline com o transformador de função
pipeline = Pipeline([
    ('log_transform', log_transformer),
    # Adicione outros estágios da sua pipeline aqui, como escalonamento, seleção de recursos, classificação, etc.
])

# Agora, você pode ajustar sua pipeline aos dados e usá-la para transformar os dados
X_transformed = pipeline.fit_transform(data[['feature']])


### Transformadores para numericos

In [ ]:
# Cria um imputer que substitui células inválidas (NaN) pela mediana dos valores da coluna à qual a célula pertence.
imputer = SimpleImputer(strategy='median')

# Antes de treinar o SimpleImputer, remover a coluna de dados categóricos. O dataset resultante tem apenas
# as variáveis independentes numéricas.
dataset_num = dataset.drop("ocean_proximity", axis=1)

# Agora treinar o Imputer. Isto vai causar o cálculo da mediana de cada coluna, 
# que ficará armazenado no Imputer para uso futuro. 
imputer.fit(dataset_num)

# O Imputer agora tem as estatísticas desejadas armazenadas.
print("Estatísticas do Imputer:")
print(imputer.statistics_)

# Compare com as medianas do DataFrame:
print("Medianas")
print(dataset_num.median().values)

In [ ]:
# Aplicar o Imputer aos nossos dados. O valor de retorno é um ndarray do NumPy.
temp = imputer.transform(housing_num)
print(type(temp))

# Trabalhar com DataFrames geralmente é mais legal - dá para referenciar colunas por nome, ao invés de indices.
# Vamos transformar o ndarray em DataFrame.
housing_tr = pd.DataFrame(temp, columns=dataset_num.columns)
print(type(housing_tr))

### Tranformadores para categoricos

In [ ]:
ordinal_encoder = OrdinalEncoder()

housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

Quando usar o OneHotEncoder em regressão linear, lembrar de fazer o $encoder = OneHotEncoder(categories='auto', drop='first')$

In [ ]:
encoder = OneHotEncoder(categories='auto', drop='first')

# Aprende a codificação e já aplica a mesma ao dataset fornecido. Todo transformador no sklearn
# tem os métodos fit() para aprender a transformação, e transform() para aplicá-la.
# O método fit_transform() faz os dois atos em sequência.
housing_cat_1hot = encoder.fit_transform(housing_cat)

# O resultado da codificação é uma matriz esparsa em NumPy.
print(housing_cat_1hot)

In [ ]:
# Convertendo em matriz densa só para observar melhor:
print(housing_cat_1hot.toarray()[:5])

# Você poderia também ter usado sparse=False na criação do OneHotEncoder.

In [ ]:
encoder.categories_

## Criando uma pipeline

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)
housing_num_tr

In [ ]:
cat_pipeline = Pipeline([
        ('cat_encoder', OneHotEncoder(sparse=False)),
    ])

housing_cat_tr = cat_pipeline.fit_transform(housing_cat)
housing_cat_tr

## Aplicando pipelines em paralelo

Pode-se organizar ColumnTransformer e Pipeline em diversas arquiteturas, em paralelo ou sequenciais. A principal diferença é que ColumnTransformer é paralelo e Pipeline é sequencial.

In [ ]:
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", cat_pipeline, cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [ ]:
housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared[:5]

## Construindo modelos preditivos

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)

In [ ]:
# Seleciona 5 pontos do conjunto de treinamento.
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]

# Prepara os dados - não se esqueça deste passo.
some_data_prepared = full_pipeline.transform(some_data)

# Para obter as previsões, basta chamar o método predict()
predicted_labels = lin_reg.predict(some_data_prepared)
print("Predição: {}".format(predicted_labels))

# Compare com os valores originais:
print("Original: {}".format(some_labels.values))

## Calculando o erro de predição

In [ ]:
from sklearn.metrics import mean_squared_error

housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print('Regressão linear: RMSE = {:.2f}'.format(lin_rmse))

## Melhorando nossa avaliação usando validação cruzada

In [ ]:
from sklearn.model_selection import cross_val_score

lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)


def display_scores(scores):
    print("Scores (ordenados): [{}]".format(" ".join(["{:.2f}".format(x) for x in sorted(scores)])))
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(lin_rmse_scores)

## Comparando modelos com teste de hipótese paramétrico

Provavelmente não será usado na prova

In [ ]:
from scipy.stats import ttest_ind

def compara_scores(scores_1, scores_2):
    t_stat, p_value = ttest_ind(scores_1, scores_2, equal_var=False)
    print("Valor da estatística t: {:.2f}".format(t_stat))
    print("Valor-p: {}".format(p_value))

compara_scores(forest_rmse_scores, tree_rmse_scores)

## Ajustando hiperparametros

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import GridSearchCV
from timeit import default_timer

param_grid = [
    # try 6 (2×3) combinations of hyperparameters
    {'n_estimators': [10, 30], 'max_features': [4, 6, 8]},
    # then try 4 (1x2×2) combinations with bootstrap set as False
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=RANDOM_SEED)

# train across 5 folds, that's a total of (6+4)*5=50 rounds of training 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)

grid_search.fit(housing_prepared, housing_labels)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

## Entendendo a importância das features

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

Para saber quem-é-quem:

In [ ]:
extra_attribs = ["rooms_per_hhold", "pop_per_hhold", "bedrooms_per_room"]
cat_encoder = cat_pipeline.named_steps["cat_encoder"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

## Medir o desempenho final

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

print("RMSE = {}".format(final_rmse))

## GridSearch para encontrar melhor alpha no Lasso

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import numpy as np

# Crie um conjunto de dados fictício para fins de exemplo
# X e y representam as características e os rótulos, respectivamente
np.random.seed(0)
X = np.random.rand(100, 10)
y = np.random.rand(100)

# Defina os valores de alpha que você deseja testar
alphas = [0.001, 0.01, 0.1, 1, 10]

# Crie um modelo de Lasso Regression
lasso = Lasso()

# Crie um dicionário com os parâmetros que você deseja ajustar
param_grid = {'alpha': alphas}

# Use GridSearchCV para encontrar o melhor valor de alpha
grid_search = GridSearchCV(lasso, param_grid, cv=5)  # cv é o número de dobras na validação cruzada

# Ajuste o modelo aos dados
grid_search.fit(X, y)

# Imprima o melhor valor de alpha encontrado
print("Melhor valor de alpha encontrado:", grid_search.best_params_['alpha'])


## GridSearch para encontrar melhor alpha e $r$ no ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
import numpy as np

# Crie um conjunto de dados fictício para fins de exemplo
# X e y representam as características e os rótulos, respectivamente
np.random.seed(0)
X = np.random.rand(100, 10)
y = np.random.rand(100)

# Defina os valores de alpha e r que você deseja testar
alphas = [0.001, 0.01, 0.1, 1, 10]
rs = [0.1, 0.5, 0.7, 0.9]

# Crie um modelo de ElasticNet Regression
elastic_net = ElasticNet()

# Crie um dicionário com os parâmetros que você deseja ajustar
param_grid = {'alpha': alphas, 'l1_ratio': rs}

# Use GridSearchCV para encontrar os melhores valores de alpha e r
grid_search = GridSearchCV(elastic_net, param_grid, cv=5)  # cv é o número de dobras na validação cruzada

# Ajuste o modelo aos dados
grid_search.fit(X, y)

# Imprima os melhores valores de alpha e r encontrados
print("Melhor valor de alpha encontrado:", grid_search.best_params_['alpha'])
print("Melhor valor de r encontrado:", grid_search.best_params_['l1_ratio'])
